Achintya Yedavalli

# Assignment 5: Text Classification

We are going work on the task of fake news detection from Tweets, i.e.,given a Tweet related Covid19, classify whether the tweet is fake or not. You are given two data files: `FakeNews_train.csv` and `FakeNews_test.csv`. These files contain two columns, "tweet" and "label," where "label" indicates whether a tweet is "real" or "fake". Your task is to explore machine learning-based models to select the best model for classifying fake news tweets.

## 1. Data Preprocessing

### a) Load the raw training & test datasets from the csv files

In [8]:
# load de data
import pandas as pd

train = pd.read_csv("FakeNews_train.csv")
test = pd.read_csv("FakeNews_test.csv")
train.head()

,tweet,label
0,• Businesses and offices can reopen for staff ...,real
1,RT @CDCDirector: We do not know yet if the ant...,real
2,This is an image of a suspected coronavirus va...,fake
3,We can’t forget that in the middle of a global...,fake
4,#IndiaFightsCorona Focused and effective effor...,real


### b) Clean the text data by removing URLs, hashtags, and mentions.

In [9]:
# cleaning time!
# we use regex for this
import re
# credit to https://www.geeksforgeeks.org/remove-urls-from-string-in-python/
def remove_non_english(text):
    # Define a regex pattern to find
    pattern = re.compile(r"https?://\S+|(?<=\s)[@#]|^[@#]|[^a-zA-Z0-9\s]")

    # Use the sub() method to replace
    text_without_noneg = pattern.sub("", text)

    return text_without_noneg

train_clean = []
test_clean = []
for line in train["tweet"]:
  train_clean.append(remove_non_english(line))

for line in test["tweet"]:
   test_clean.append(remove_non_english(line))

# make my life easier by putting back clean into the original dfs
train["tweet"] = train_clean
test["tweet"] = test_clean 

### c) Tokenize the text data.

### d) Remove stop words and lemmatize the text data.

(doing c and d together because it's easier)

In [10]:
#tokenize data
import spacy
from nltk.corpus import stopwords
nlp_pipeline = spacy.load("en_core_web_sm")

# get a list of stopwords from NLTK
import nltk
nltk.download('stopwords')
stops = set(stopwords.words('english'))

def pre_process_a_single_sentence(sentence: str):
    # Lower case text
    sentence = sentence.lower()

    processed_sentence = []

    # Tokenize, and lemmatize the text
    doc = nlp_pipeline(sentence)

    for token in doc:
      # here token is an object that contains various information about each token
      # information such as lemma, pos, parse labels are available
      # we will check here if tokens are present in stopwords; if not, we will retain their lemma
      if token not in stops:
        lemmatized_token = token.lemma_
        processed_sentence.append(lemmatized_token)
    processed_sentence = " ".join (processed_sentence)
    return processed_sentence

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
train_clean = []
test_clean = []

# run the pre-processing
for tweet in train["tweet"]:
    train_clean.append(pre_process_a_single_sentence(tweet))

for tweet in test["tweet"]:
    test_clean.append(pre_process_a_single_sentence(tweet))

train["tweet"] = train_clean
test["tweet"] = test_clean

train_clean = None
test_clean = None

### e) Save the processed data into 2 files

In [16]:
train.to_csv("FakeNews_train_preprocessed.csv")
test.to_csv("FakeNews_test_preprocessed.csv")